In [165]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

LC = pd.read_csv("C:\\Users\\skyamz\\Desktop\\ppdai_3_213\\LC.csv") 
LP = pd.read_csv("C:\\Users\\skyamz\\Desktop\\ppdai_3_213\\LP.csv")

In [166]:
#把日期都转换程日期格式，方便计算
LP['到期日期'] = pd.to_datetime(LP['到期日期'])
LP['还款日期'] = LP['还款日期'].replace('\\N','2017-02-22')#将还款日期不明确的标的，还款日期统一设置为记录日期，即2017-02-22                
LP['还款日期'] = pd.to_datetime(LP['还款日期'])
LC['orn_rank'] = LC['初始评级']#为了防止中文乱码的问题，把’初始评级‘改成’orn_rank‘

In [167]:
#计算逾期，并将其转换成int格式，为了以后计算方便
LP['day_diff'] =  LP['还款日期'] - LP['到期日期']
LP['day_diff'] = LP['day_diff'].astype(np.str) #先转换成string格式，然后利用map函数来逐一变成数字
Tran_Date = lambda day_diff: int(day_diff.split('days',1)[0])
LP['day_diff']= LP['day_diff'].map(Tran_Date)

In [168]:
LP02 = LP[(LP['还款状态']==2)|(LP['还款状态']==0)]#选择已经逾期的标的和未还款的标的

In [169]:
LP02 = LP02[(LP02['day_diff']<=90)&(LP02['day_diff']>=1)] #催收回款的定义为逾期 90 天之内的逾期还款                                        

In [170]:
LP02 = pd.merge(LC,LP02,on='ListingId')#内联LC与LP

In [171]:
#根据初始评级和逾期，对逾期和未还的标的进行分类
lable = ['orn_rank','day_diff']
level = pd.DataFrame(LP02.pivot_table(index=lable,columns=['还款状态'],values=['应还本金'],aggfunc=np.sum))

In [172]:
level.columns= ['0','2']
level = level.fillna(0)
#计算催收回款率（back_rate） = 催收回本金(2)/所有逾期本金(total)
level['total'] = level['0'] + level['2']
level['back_rate'] = level['2']/level['total']
level['back_rate'] = level['back_rate'].astype(np.str)#将back_rate变为str，方便输出
level = level.reset_index()#将level标中的标签变换为列名

In [173]:
rank  = list(level['orn_rank'].unique())#取初始评级
day_diff,back_rate = [],[]
#根据不同的初始评级，将逾期和催收回款率存入list中，方便画图
for i in rank:
        temp = level[level['orn_rank']==i]
        day_diff.append(list(temp['day_diff']))
        back_rate.append(list(temp['back_rate']))

In [174]:
back_rate

[['0.987102690632',
  '0.979973389615',
  '0.981874356994',
  '0.972461397802',
  '0.970252301412',
  '0.946713593825',
  '0.974347509563',
  '0.98596843521',
  '0.962145937735',
  '0.966921976392',
  '0.927844102413',
  '0.949627344791',
  '0.984344270844',
  '0.970730900283',
  '0.985764912216',
  '0.863996596931',
  '0.933362565978',
  '0.753113790925',
  '0.967029704005',
  '0.984633291518',
  '0.994873776638',
  '0.998267628371',
  '0.924422990124',
  '0.969626013071',
  '0.791681817325',
  '0.921794411766',
  '1.0',
  '0.93005126784',
  '0.91012386512',
  '0.97337641924',
  '0.98393813061',
  '0.803526298753',
  '0.971876609817',
  '0.93006920376',
  '0.924608498566',
  '0.920339130639',
  '0.662344575673',
  '0.965925604856',
  '0.997715343309',
  '0.858603571629',
  '0.187278376674',
  '0.948901684892',
  '0.988004331219',
  '0.973403761812',
  '0.9635097617',
  '0.995783013755',
  '0.929683159305',
  '0.802690826429',
  '0.647647990913',
  '0.849905526689',
  '0.799430639781',

In [175]:
#绘制初始评级对催收回款率影响的折线图
fig = plt.figure(figsize = (20,10))#设置大小
plt.title('orn_rank influnence')#标题
plt.xlabel('day_diff')#X轴
plt.ylabel('back_rate')#Y轴
#分别对应['A','B','C','D','E','F'] 的颜色
colors = ['red','blue','green','orange','black','purple']
#对于每个初始评级，绘制它们对应的催收回款率
for i in range(0,6):
        plt.plot(day_diff[i],back_rate[i],c = colors[i],label = rank[i])
        plt.legend(loc='lower left')#标签位置

In [176]:
plt.show()